## Visual Acuity Test Assesment Model

Model Used: Random Forest Classifier

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

In [4]:
data = pd.read_csv('/content/visual_acuity_data.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UserID                  400 non-null    int64  
 1   Age                     400 non-null    int64  
 2   Gender                  400 non-null    object 
 3   Accuracy (%)            400 non-null    float64
 4   ResponseTime (ms)       400 non-null    float64
 5   NumCorrectLetters       400 non-null    float64
 6   EyesightClassification  400 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 22.0+ KB


In [6]:
data.head()

,UserID,Age,Gender,Accuracy (%),ResponseTime (ms),NumCorrectLetters,EyesightClassification
0,1,46,Male,61.85,1253.0,12.0,Bad
1,2,39,Male,61.23,1664.0,12.0,Bad
2,3,45,Male,79.75,1733.0,16.0,Normal
3,4,27,Male,91.88,1340.0,18.0,Good
4,5,25,Male,59.48,1615.0,12.0,Bad


In [7]:
print(data['EyesightClassification'].value_counts())

EyesightClassification
Bad       166
Normal    123
Good      111
Name: count, dtype: int64


In [8]:
features = data[['Accuracy (%)', 'ResponseTime (ms)', 'NumCorrectLetters']]
target = data['EyesightClassification']

In [9]:
le = LabelEncoder()
target = le.fit_transform(target)

target = pd.Series(target)

print(target.value_counts())

0    166
2    123
1    111
Name: count, dtype: int64


In [10]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
scaled_features_df = pd.DataFrame(scaled_features, columns=features.columns)

X_train, X_temp, y_train, y_temp = train_test_split(scaled_features_df, target, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(240, 3)
(96, 3)
(64, 3)


In [12]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [13]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')

print(f"Cross-Validation Scores: {cv_scores}")
print(f"\nMean CV Score: {np.mean(cv_scores)}")

Cross-Validation Scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

Mean CV Score: 1.0


In [15]:
y_pred_val = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred_val)
print(f"Validation Accuracy: {accuracy}")
print("\nValidation Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))
print("\nValidation Classification Report:\n", classification_report(y_val, y_pred_val))

Validation Accuracy: 0.984375

Validation Confusion Matrix:
 [[28  0  0]
 [ 0 14  1]
 [ 0  0 21]]

Validation Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       1.00      0.93      0.97        15
           2       0.95      1.00      0.98        21

    accuracy                           0.98        64
   macro avg       0.98      0.98      0.98        64
weighted avg       0.99      0.98      0.98        64



In [16]:
y_pred_test = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy}")
print("\nTest Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("\nTest Classification Report:\n", classification_report(y_test, y_pred_test))

Test Accuracy: 0.9895833333333334

Test Confusion Matrix:
 [[42  0  1]
 [ 0 35  0]
 [ 0  0 18]]

Test Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99        43
           1       1.00      1.00      1.00        35
           2       0.95      1.00      0.97        18

    accuracy                           0.99        96
   macro avg       0.98      0.99      0.99        96
weighted avg       0.99      0.99      0.99        96



In [17]:
pickle.dump(model, open('visual_acuity_model.pkl', 'wb'))